<h1>0. Packages</h1>

In [1]:
from importlib import reload
import numpy as np
import re
import skyfield.sgp4lib as spg4
from   skyfield.api import wgs84
import matplotlib 
import PyQt5
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.plotly as plotly

# User defined lib.
import generate_debris as gd
import CoordTransforms as ct
import visualize_deb as vis
import planetary_data

reload(gd)
reload(ct)

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.stats import uniform
from scipy.stats import norm
import scipy.stats as stats
from enum import IntEnum    
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from itertools import product, combinations
from matplotlib import cm
from scipy import integrate
import pandas as pd
from numba import njit, prange
from skyfield.api import load, EarthSatellite, wgs84

%matplotlib qt5

debris_category = IntEnum('Category', 'rb sc soc')

<IPython.core.display.Javascript object>

In [14]:
import os

# Create Directory to Save Figures to
if not os.path.exists("figures"):
    os.mkdir("figures")
    
# Retreiving API Keys from OS
PLOTLY_API_KEY = os.environ.get('PLOTLY_API_KEY')
PLOTYLY_USERNAME = 'rhumphreys2017'

In [15]:
# Loging into chart studio
import chart_studio
chart_studio.tools.set_credentials_file(username=PLOTYLY_USERNAME, api_key=PLOTLY_API_KEY)

<h1>1. Data Structure</h1>

<h3>1.1 Satellite Structure</h3>

Creating an object to represent satellite from 3le.txt <br>
May end up using implementation from another package

<b>Data sources:<b><br>
https://www.space-track.org    
https://www.celestrak.com/NORAD/documentation/spacetrk.pdf

<b>Information about 3le/2le:</b><br>
https://en.wikipedia.org/wiki/Two-line_element_set#cite_note-nasahelp-12
https://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/SSOP_Help/tle_def.html

In [ ]:
# class two_line_element:
    
#     def __init__(self, data):
#         self.parse_data(data)
        
#     def parse_data(self, satelite_data):
#         self.title = satelite_data[0]
#         line_one = satelite_data[1].split()
#         line_two = satelite_data[2].split()

#         self.catalog_number = re.search('[0-9]{1,5}', line_one[0]).group()
#         self.classification = re.search('[UCS]', line_one[0]).group()

#         international_designator = re.search('([0-9]{2})([0-9]{3})(\w{1,3})', line_one[1])
#         self.launch_year = international_designator.group(1)
#         self.launch_number = international_designator.group(2)
#         self.launch_piece =  international_designator.group(3)

#         epoch = re.search('([0-9]{2})(\d+\.\d+)', line_one[2])
#         self.epoch_year = epoch.group(1)
#         self.epoch_day = epoch.group(2)

#         self.ballistic_coefficient = float(line_one[3])
#         self.mean_motion_double_prime = line_one[4]
#         self.BSTAR = line_one[5]

#         self.inclination = line_two[1]
#         self.right_ascension_of_ascending_node = line_two[2]
#         self.eccentricity = line_two[3]
#         self.argument_of_perigee = line_two[4]

#         mean_anomaly = line_two[5]

<h3>Data structure implementation using spg4 from skyfield library<h3>

In [ ]:
# Opening the .txt file
with open("3le.txt") as f:
    txt = f.read()
    
sat_lines = re.findall('(.*?)\n(.*?)\n(.*?)\n', txt)

# Preview of `sat_lines`, each element should be the three lines representing a satellite
print(sat_lines[0])

# Convert each group of 3 lines into a satelite object
def line_element_to_satellite(lines):
    title = lines[0]
    line1 = lines[1]
    line2 = lines[2]
    return spg4.EarthSatellite(line1, line2, name=title)

satellites = [line_element_to_satellite(lines) for lines in sat_lines]

# Retrieving orbital structure that fits criterion
i_desired = np.deg2rad(20)
z_desired = 700*1e3 #[m]

i_tol = np.deg2rad(5)
z_tol = 25*1e3

data = np.array(satellites)
ts = load.timescale(builtin=True)
t  = ts.now()
r_E = 6.3781e6 #[m]


inclo = np.array([sat.model.inclo for sat in data ])
altitude = np.array([wgs84.subpoint(sat.at(t)).elevation.m for sat in data])

I_inclo = (inclo < i_desired + i_tol) & (inclo > i_desired - i_tol)
I_alt   = (altitude < z_desired + z_tol) & (altitude > z_desired - z_tol)

# Getting the first satellite that meets the above criterion
satellite = data[I_inclo & I_alt][0] 

<h3>1.2 Celestial Bodies</h3>


de421.bsp is a Ephemerides provided by JPL Horizon which has the calculated positions of celestial bodies within a certain time interval. de421 is commonly used due to its small size and its relativley up to date information

In [ ]:
# from skyfield.api import load, EarthSatellite
# ts = load.timescale(builtin=True)

# # Loading the data from de421.bsp using skyfield 
# planets = load('de421.bsp')

# # Finding the data about Earth
# earth = planets["Earth"]

<h1>2. Breakup Models</h1>

<h3>2.1 NASA breakup model</h3>

<b>Implementation:</b> (Found on page 207)<br>
https://www.researchgate.net/publication/295490674_Space_debris_cloud_evolution_in_Low_Earth_Orbit

<b>Alternate implementation:</b><br>
https://gitlab.obspm.fr/apetit/nasa-breakup-model/tree/master

<b>Information:</b><br>
https://digitalcommons.calpoly.edu/cgi/viewcontent.cgi?referer=https://en.wikipedia.org/&httpsredir=1&article=1094&context=theses


<b>Now implemented in file generate_debris.py</b><br>

<h3>2.2 NASA breakup model validation</h3>

In [ ]:
# Explosion Case

reload(gd)

m_target = 1000 # [kg]

# Do not need to specify projectile mass, and collision velocity for an explosion
L_c, areas, masses, AM = gd.fragmentation(m_target, 0, 0, False, debris_category.rb, True)

bins = [1e-3, 1e-2, 1e-1, 1, np.inf] # L_c bins in meters
h,b = np.histogram(L_c, bins=bins)

ch = [ { f'>{bins[i]}m':np.sum(h[i:]) for i in range(len(h))}]

df = pd.DataFrame(ch)
df

In [ ]:
# reload(gd)

# %matplotlib inline

# L_c, areas, masses, AM = gd.fragmentation(1000, 10, 10, True, debris_category.rb, False)
# N_fragments_total = L_c.shape[0]
# lambda_c = np.log10(L_c)

# def create_log_bins(values, nbins=100):
#     #return np.geomspace(values.min(), values.max(), nbins)
#     bins = np.geomspace(values.min(), values.max(), nbins)
#     a = bins[1]/bins[0]
#     bins = np.concatenate([[bins[0]/a], bins,[bins[-1]*a]])
#     return bins

# # # # Validating L_c
# # h, b = np.histogram(L_c, bins=create_log_bins(L_c))
# # plt.figure(figsize=(5,5))
# # plt.xscale('log')
# # plt.xlabel(r'$L_{c}$')
# # plt.ylabel(r'$N_f$')
# # plt.gca().set_xticks([1e-3, 1e-2, 1e-1, 1, 10])
# # plt.gca().set_xlim([1e-3,10])
# # plt.ticklabel_format(axis="y", style="sci", scilimits=(0,5))
# # plt.gca().set_yticks([0, 2e5, 4e5, 6e5, 8e5])
# # plt.gca().set_ylim([0,8e5])
# # plt.plot((b[:-1] + b[1:])/2, h, '.-')
# # plt.show()

# # # # Validating Areas
# # h, b = np.histogram(areas, bins=create_log_bins(areas))
# # plt.figure(figsize=(5,5))
# # plt.xscale('log')
# # plt.xlabel(r'$Areas$')
# # plt.ylabel(r'$N_f$')
# # plt.gca().set_xticks([1e-8,1e-6, 1e-4, 1e-2, 1, 1e2])
# # plt.gca().set_xlim([1e-8,1e2])
# # plt.ticklabel_format(axis="y", style="sci", scilimits=(0,5))
# # plt.gca().set_yticks([0, 2e5, 4e5, 6e5, 8e5])
# # plt.gca().set_ylim([0,8e5])
# # plt.plot((b[:-1] + b[1:])/2, h, '.-')
# # plt.show()

# # # Validating Mass & Velocity
# # h, b = np.histogram(masses, bins=create_log_bins(masses))
# # plt.figure(figsize=(5,5))
# # plt.xscale('log')
# # plt.xlabel(r'$Masses$')
# # plt.ylabel(r'$N_f$')
# # plt.gca().set_xticks([1e-8, 1e-5, 1e-2, 10, 1e4])
# # plt.gca().set_xlim([1e-8,1e4])
# # plt.ticklabel_format(axis="y", style="sci", scilimits=(0,5))
# # plt.gca().set_yticks([0, 2e5, 4e5, 6e5, 8e5])
# # plt.gca().set_ylim([0,8e5])
# # plt.plot((b[:-1] + b[1:])/2, h, '.-')
# # plt.show()

# # # # Validating Velocity
# # deltaV = np.array(gd.distribution_deltaV(AM, 10, False))
# # h, b = np.histogram(deltaV, bins=create_log_bins(deltaV))
# # plt.figure(figsize=(5,5))
# # plt.xscale('log')
# # plt.xlabel(r'$\delta V$')
# # plt.ylabel(r'$N_f$')
# # plt.gca().set_xticks([1e-3, 1e-2, 1e-1, 1, 10])
# # plt.gca().set_xlim([1e-3,10])
# # plt.grid()
# # plt.plot((b[:-1] + b[1:])/2, h, '.-')
# # plt.ticklabel_format(axis="y", style="sci", scilimits=(0,5))
# # plt.gca().set_yticks([0, 2e5, 4e5, 6e5, 8e5])
# # plt.gca().set_ylim([0,8e5])
# # plt.show()

<h3>2.3 and Onward will be alternate breakup models</h3>

<h1>3. Numerical Propagation</h1>

<h3>3.2 Calculating Orbital Debris Properties</h3>

In [10]:
from skyfield.api import load, EarthSatellite
reload(gd)

ts = load.timescale(builtin=True)

t_fragmentation = ts.now()
geocentric      = satellite.at(t_fragmentation)
init_position   = geocentric.position.m

m_target         = 250   # [kg] (Approx. mass of starlink sat)
m_projectile     = 200     # [kg]
v_impact         = 2   # [km·s^-1] (Measured relative to the target) (Needs to be in km·s^-1)
is_catastrophic  = False
is_explosion     = False

L_c, areas, masses, AM = gd.fragmentation(m_target, m_projectile, v_impact, is_catastrophic, debris_category.sc, is_explosion)
deltaV = np.array(gd.distribution_deltaV(AM, v_impact, False)) # Returns as [km·s^-1]
deltaV = deltaV * 1e3    #[m·s^-1]

<h3>3.3 Converting Cartesian to Keplerian</h3>

In [ ]:
from numpy.linalg import norm
init_position = geocentric.position.m

reload(planetary_data)
reload(ct)

deb_positions       = np.empty((len(AM), 3))
deb_positions[:, :] = init_position[None,:]
deb_velocities      = gd.velocity_vectors(len(AM), geocentric.velocity.m_per_s, deltaV)

keplerian_state     = ct.rv2coe(deb_positions, deb_velocities, planetary_data.earth['mu'])

print(keplerian_state.shape)
# Removing fragments that would renter earth
periapsis           = keplerian_state[0, :] * (1 - keplerian_state[1, :])
I                   = np.argwhere(periapsis > planetary_data.earth['radius'])
keplerian_state     = np.squeeze(keplerian_state[:, I])
areas  = areas[I].flatten()  # When doing the indexing , a 1d dim being added which is unneccesary
masses              = masses[I].flatten() # When doing the indexing , a 1d dim being added which is unneccesary

In [ ]:
# rand_index = 1

# print("Cartesian")
# print("r: ", deb_positions[rand_index, :])
# print("v: ", deb_velocities[rand_index, :])
# print("m: ", masses[rand_index])
# print("m_E: ", planetary_data.earth['mass'])
# print("Keplerian")
# ks_rand = keplerian_state[:, rand_index]
# print("a: ", ks_rand[0]) # a is tyically off in the thousands place
# print("e: ", ks_rand[1])
# print("i: ", ks_rand[2])
# print("Omega: ", ks_rand[3]) 
# print("omega: ", ks_rand[4])
# print("M: ", ks_rand[5])
# print("nu: ", ks_rand[6])
# print()

<h3>3.4 Testing coverting back to Cartesian</h3>

In [ ]:
# reload(ct)

# %matplotlib notebook

# indexes = np.random.default_rng().choice(keplerian_state.shape[1], size=1000, replace=False)
# ks_sample = keplerian_state[:, indexes]

# cartesian_state = np.array(ct.coe2rv_many(k=planetary_data.earth['mu'],
#                               p=ks_sample[7, :],
#                               ecc=ks_sample[1, :],
#                               inc=ks_sample[2, :],
#                               raan=ks_sample[3, :],
#                               argp=ks_sample[4, :],
#                               nu=ks_sample[6, :]))
# radius_earth = planetary_data.earth['radius']
# r = cartesian_state[0, :, :]
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(r[:, 0], r[:, 1], r[:, 2])
# ax.set_xlim3d([-radius_earth-1000*1e3, radius_earth+1000*1e3])
# ax.set_xlabel('X (m)')

# ax.set_ylim3d([-radius_earth-1000*1e3, radius_earth+1000*1e3])
# ax.set_ylabel('Y (m)')

# ax.set_zlim3d([-radius_earth-1000*1e3, radius_earth+1000*1e3])
# ax.set_zlabel('Z (m)')

# _u, _v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
# _x = radius_earth*np.cos(_u)*np.sin(_v)
# _y = radius_earth*np.sin(_u)*np.sin(_v)
# _z = radius_earth*np.cos(_v)

# ax.plot_surface(_x, _y, _z, cmap=cm.coolwarm)
# plt.show()

In [ ]:
# keplerian_state.shape
# masses.shape

<h3>3.4 Debris Cloud formation</h3>

In [ ]:
## Shrink number of debris being used
indexes = np.random.default_rng().choice(keplerian_state.shape[1], size=1000, replace=False)

ks = keplerian_state[:, indexes]
masses = masses[indexes]
areas = areas[indexes]
deb_positions = deb_positions[indexes, :]
deb_velocities = deb_velocities[indexes, :]

In [ ]:
np.savez("debris.npz",
         keplerian_states=ks,
         masses=masses,
         areas=areas,
         deb_positions=deb_positions,
         deb_velocities=deb_velocities)

In [4]:
container = np.load('debris.npz')
ks = container['keplerian_states']
masses = container['masses']
areas = container['areas']
deb_positions = container['deb_positions']
deb_velocities = container['deb_velocities']

<h4>3.4.1 Ellisoid formation</h4>

In [5]:
import Perturbations as OP
import planetary_data as pd
from Perturbations import null_perts

reload(OP)

# Cleanup states to remove any fragments that would deorbit, given no perturbations
periapsis     = ks[0, :] * (1 - ks[1, :])
I             = np.argwhere(periapsis > pd.earth['radius'])
ks_pruned     = np.squeeze(ks[:, I])
T             = ks_pruned[8, :]
areas_pruned  = areas[I].flatten()  # When doing the indexing , a 1d dim being added which is unneccesary
masses_pruned = masses[I].flatten() # When doing the indexing , a 1d dim being added which is unneccesary

# Propagate orbit for a period of time
perts = null_perts()
perts['aero'] = True
perts['J2']   = True
op = OP.OrbitPropagator(ks_pruned, areas_pruned, masses_pruned, [0, 40*np.ceil(max(T))], 60*5, perts=perts)
op.propagate_orbit()

# Get the cartesian state representation
cartesian_states = op.cartesian_representation()

<h4>3.4.2 Particle Debris flux</h4>

Using a particle flux to determine when the fragments of the debris have finished the formation of the ring. Indicating the end of the first phase of the debris cloud formation. This is accomplished by creating an xz plane and detecting when particles have switched from one side to the other. This approach will cause a peak as fragments pass through that becomes uniform as the debris becomes uniformly spread out.

In [6]:
import time
import matplotlib.dates as mdates
from dateutil import tz

def fragmentation_flux(X):
    return np.sum((X[:-1, :, 1] < 0) & (X[1:, :, 1] > 0), axis=1)
    
position = cartesian_states[:, 0, :, :]
flux = fragmentation_flux(position)

<h4>3.4.3 Convergence of the flux</h4>

The next step is determining when the fragments have ended the torroid formation phase. This occurs when the fragments are approximately uniformally spread out. We can check to see when the flux meets a convergence criterion to determine when this happens.

In [7]:
w = 100 # Window of points to look at
tol = 1.5
convergence_ratio = np.array([np.var(flux[i:i+w])/np.mean(flux[i:i+w]) for i in range(len(flux))])    
intersection_index = np.argwhere(convergence_ratio <= tol).flatten()[2] # Grabbing the second point that meets the criteria

# datetimes
t_flux = t_fragmentation.utc_datetime() + np.array(range(len(flux))) * datetime.timedelta(minutes = 5)

# Removing last window from `t_flux`, `flux`, and `convergence_ratio` bc. not well defined for last values
t_flux = t_flux[:-w]
flux = flux[:-w]
convergence_ratio = convergence_ratio[:-w]

In [41]:
import datetime
import chart_studio.plotly as py

# SIgn in 
py.sign_in(username = PLOTYLY_USERNAME, api_key = PLOTLY_API_KEY)

# Creating Flux v. Time plot
layout = go.Layout(
    title        = dict(text='$\\text{Flux}\:(\\Phi)\:\\text{vs. Time }(t)$',
                        x=0.5),
    xaxis_title  = '$t\:[days]$',
    yaxis_title  = '$\\text{ Number of fragments passing XZ plane, }\Phi\:$',
    template     = 'plotly_white'
)


data = [
    go.Scatter(x=t_flux, y=flux,
               mode='lines',
               name='Flux'),
    go.Scatter(x=[t_flux[intersection_index], t_flux[intersection_index]], y=[0, np.max(flux)],
               mode='lines',
               line=dict(dash = 'dash'),
               name='Convergence')
]

fig1 = go.Figure(data=data, layout=layout)

# Stopping data to have half before intersection index and half after
index_stop = intersection_index * 2
if index_stop > len(flux) - 1 : index_stop = len(flux) - 1
fig1.update_layout(xaxis_range=[t_flux[0],t_flux[index_stop]])

# Saving plot as an image and uploading it to plotly
fig1.write_image("figures/Flux_v_Time.png", width=500, height=500, scale=2)
py.iplot(fig1, filename="Flux v. Time")

In [37]:
intersection_index

579

In [26]:
#Creating Convergence Ratio v. Time plot
layout = go.Layout(
    title        = dict(text='Convergence ratio vs. Time (t)',
                        x=0.5),
    xaxis_title  = '$t\:[days]$',
    yaxis_title  = 'Convergence ratio []',
    template     = 'plotly_white',
    legend       = go.layout.Legend(yanchor="top",
                             y=0.99,
                             xanchor="right",
                             x=0.99)
)
data = [
    go.Scatter(x=t_flux, y=convergence_ratio,
               mode='lines',
               name='Convergence ratio'),
    go.Scatter(x=[t_flux[intersection_index], t_flux[intersection_index]], y=[0, np.max(flux)],
               mode='lines',
               line=dict(dash = 'dash'),
               name='Convergence time'),
    go.Scatter(x=[t_flux[0], t_flux[-1]], y=[tol, tol],
               mode='lines',
               line=dict(dash = 'dash'),
               name='Tolerance'),
]
fig2 = go.Figure(data=data, layout=layout)
fig2.update_yaxes(type="log")
fig2.write_image("figures/Convergence_Ratio_v_Time.png", width=500, height=500, scale=2)

py.iplot(fig2, filename="Convergence Ratio v. Time")

<h4>3.4.4 End of band formation</h4>

Now that the band has formed, we can shift away from propagating the exact position of each fragments and inplace propgate their changes in eccentricity and semi major axis due to drag. To do this first we must get the final states of the debris after the band has formed.

In [ ]:
cs_toroid = cartesian_states[:intersection_index, :, :, :]
ks_toroid = op.states[0:intersection_index, :, :]
op.states = ks_toroid



In [ ]:
import plotly.express as px  # (version 4.7.0)
import plotly.io as pio
import pandas
reload(vis)


spherical_earth_map = np.load('map_sphere.npy') 

pos_toroid = cs_toroid[:, 0, :, :]/1e3
N_timesteps = pos_toroid.shape[0]
N_fragments = pos_toroid.shape[1]
r_E = op.cb['radius'] / 1e3
xm, ym, zm = spherical_earth_map.T * r_E

# Converting data to pandas dataframe
df = pandas.DataFrame()
# *** Update this if chnage timestep in initial orbit propagation ***
dt = 60 * 5 #[s]
# Want to show the evolution in 30 min
timesteps = np.arange(0,N_timesteps, 6)

for t in timesteps:   
    step = t*np.ones_like(N_timesteps)
    time = dt * step / 60 #[min]
    d = {'X': pos_toroid[t, :, 0],
         'Y': pos_toroid[t, :, 1],
         'Z':pos_toroid[t, :, 2],
         'Min.': time,
         'a': ks_toroid[t, 0, :]/1e3,
         'e': ks_toroid[t, 1, :],
         'i': ks_toroid[t, 2, :],
        }
    df = pandas.concat([df, pandas.DataFrame(data=d)])

# Creating visual
def spheres(size, clr, dist=0): 
    
    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)
    
    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))
    
    # Set up trace
    trace= go.Surface(x=x0, y=y0, z=z0, colorscale=[clr, clr], opacity=0.2)
    trace.update(showscale=False)

    return trace

fig = px.scatter_3d(
    data_frame=df,
    x='X',
    y='Y',
    z='Z',
    # color_discrete_map={'Europe': 'black', 'Africa': 'yellow'},
    # opacity=0.3,              # opacity values range from 0 to 1
    # symbol='Year',            # symbol used for bubble
    # symbol_map={"2005": "square-open", "2010": 3},
    #size=3*np.ones((N_fragments)),       # size of bubble
    # size_max=50,              # set the maximum mark size when using size
    # range_z=[9,13],           # you can also set range of range_y and range_x
    #template='ggplot2',         # 'ggplot2', 'seaborn', 'simple_white', 'plotly',
                                # 'plotly_white', 'plotly_dark', 'presentation',
                                # 'xgridoff', 'ygridoff', 'gridon', 'none'
    title='Evolution of debris cloud to toroid formation',
    #labels={'Years in school (avg)': 'Years Women are in School'},
    hover_data={'Min.': False, 'X': False, 'Y':False, 'Z':False, 'a':':.1f', 'e':':.4f','i':':.1f' },
    #hover_name='Orbital Elements',        # values appear in bold in the hover tooltip
    height=800,                 # height of graph in pixels
    width =800,
    animation_frame='Min.',   # assign marks to animation frames
    range_x=[-r_E - 1000,r_E + 1000],
    range_z=[-r_E - 1000,r_E + 1000],
    range_y=[-r_E - 1000,r_E + 1000],

)
fig.update_traces(marker={'size': 3})
# Add Earth
earth=spheres(r_E, '#ffffff', 0) # Earth
#fig.add_trace(go.Scatter3d(x=xm, y=ym, z=zm, mode='lines', line=dict(color=zm, colorscale='Viridis')))
fig['layout']['scene']['aspectmode'] = 'cube'
fig.update_layout(transition = {'duration': 2000})
fig.show()

<h3> 4.0.0 Implementing Perturbations </h3>

<h4> 4.1.0 atmospheric model </h4>

In [ ]:
# import Aerodynamics as aero

# upper_bound = 900 * 1e3                         #[m]
# altitudes   = np.arange(0, upper_bound, 25)      #[m]
# rho         = aero.atmosphere_density(altitudes) #[kg·m^-3]

# I_standard = np.argwhere(altitudes/1e3 == 25).flatten()[0]
# I_cira    = np.argwhere(altitudes/1e3 == 500).flatten()[0]

# # Plotting the Exponential Atmospheric Model

# layout = go.Layout(
#     title        = go.layout.Title(text='Altitude (z) vs. Atmospheric Density (ρ)',
#                                    x=0.5),
#     xaxis_title  = 'z [km]',
#     yaxis_title  = '$\log_{10}(\\rho\:[kg·m^{-3}])$',
#     template     = 'plotly_white',
#     legend       = go.layout.Legend(yanchor="top",
#                              y=0.99,
#                              xanchor="right",
#                              x=0.99)
# )

# data = [
#     go.Scatter(x=altitudes[:I_standard] / 1e3, y=rho[:I_standard],
#                     mode='lines',
#                     name='U.S Standard Atmosphere'),
#     go.Scatter(x=altitudes[I_standard:I_cira] / 1e3, y=rho[I_standard:I_cira],
#                     mode='lines',
#                     name='CIRA-72'),
#     go.Scatter(x=altitudes[I_cira:] / 1e3, y=rho[I_cira:],
#                     mode='lines',
#                     name='CIRA-72 with T_infinity = 1000K')
# ]

# fig = go.Figure(data=data, layout=layout)
# fig.update_yaxes(type="log")


# fig.write_image("figures/Atmospheric_Density_v_Altitude.png", width=500, height=500, scale=2)
# f2 = go.FigureWidget(fig)
# f2

# f2

<h4> 4.2.0 J2 </h4>

<h4> 4.3.0 Application of perturbations </h4>

In [ ]:
from scipy.special import iv
from scipy import integrate
import Aerodynamics as aero

op.tspan[-1] = 3600*24*365*2
op.dt = 3600*24
de, da, di, dOmega, domega, dnu, dp = op.propagate_perturbations()

In [ ]:
z = (da * (1 - de)) - op.cb['radius']
z[z<100*1e3] = 0
z.shape
plt.close()
for i in range(10):
    plt.plot(np.trim_zeros(z[i, :]))
    
plt.show()

In [ ]:
temp = np.zeros_like(da) # The params set to 0 dont matter for converting to rv
ks_propagated = np.swapaxes(np.stack([da, de, di, dOmega, domega, temp, dnu, dp, temp, temp]).T, 1, 2)
ks_final = np.concatenate([ks_toroid, ks_propagated])
op.states = ks_final

cs_final = op.cartesian_representation()

In [ ]:
import pandas as pandas
import plotly.express as px

pos_toroid = cs_final[cs_toroid.shape[0]-1:, 0, :, :]/1e3
N_timesteps = pos_toroid.shape[0]
N_fragments = pos_toroid.shape[1]
r_E = op.cb['radius'] / 1e3


# Converting data to pandas dataframe
df = pandas.DataFrame()
# *** Update this if chnage timestep in initial orbit propagation ***
dt = 60 * 5 #[s]
# Want to show the evolution in 1 day steps
timesteps = np.arange(0,N_timesteps, 5)

for t in timesteps:   
    step = t*np.ones_like(N_timesteps)
    time = step  #[day]
    d = {'X': pos_toroid[t, :, 0],
         'Y': pos_toroid[t, :, 1],
         'Z':pos_toroid[t, :, 2],
         'Day': time,
        }
    df = pandas.concat([df, pandas.DataFrame(data=d)])

def spheres(size, clr, dist=0): 

    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)

    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))

    # Set up trace
    trace= go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]])
    trace.update(showscale=False)

    return trace
fig = px.scatter_3d(
    data_frame=df,
    x='X',
    y='Y',
    z='Z',
    title='Evolution of debris cloud to Band formation',
    #labels={'Years in school (avg)': 'Years Women are in School'},
    #hover_data={'Min.': False, 'X': False, 'Y':False, 'Z':False, 'a':':.1f', 'e':':.4f','i':':.1f' },
    #hover_name='Orbital Elements',        # values appear in bold in the hover tooltip
    height=800,                 # height of graph in pixels
    width =800,
    animation_frame='Day',   # assign marks to animation frames
    range_x=[-r_E - 1000,r_E + 1000],
    range_z=[-r_E - 1000,r_E + 1000],
    range_y=[-r_E - 1000,r_E + 1000],

)
fig.update_traces(marker={'size': 3})
# Add Earth
earth=spheres(r_E, '#ffffff', 0) # Earth
fig.add_trace(earth)
#fig.add_trace(go.Scatter3d(x=xm, y=ym, z=zm, mode='lines', line=dict(color=zm, colorscale='Viridis')))
fig['layout']['scene']['aspectmode'] = 'cube'
fig.update_layout(transition = {'duration': 2000})
fig.show()

# Propagation Analysis

In [ ]:
index = int(np.ceil(ks_propagated.shape[0]*.10)) # index near begining
raan_0 = ks_propagated[index, 3, :].copy() % 360
raan_0[raan_0 > 180] -= 360 # Converting angles to new range

raan_mid = ks_propagated[ks_propagated.shape[0] // 2, 3, :].copy()  % 360
raan_mid[raan_mid  > 180] -= 360 

raan_f = ks_propagated[-1, 3, :].copy() % 360
raan_f[raan_f > 180] -= 360

In [ ]:
import math
import plotly.figure_factory as ff

uniform_dist = np.random.uniform(-180, 180, len(raan_0))
group_labels = ['raan_0', 'raan_mid', 'raan_f', 'uniform']
fig = ff.create_distplot([raan_0, raan_mid, raan_f, uniform_dist], group_labels, show_hist =  False)

# Updating the uniform curve to be dashed
index = np.argwhere(np.array([data.legendgroup for data in fig.data]) == 'uniform')[0][0]
fig.data[index].line = dict(color='red', width=2,
                             dash='dash')

fig.show()